# Self-Driving Car Engineer Nanodegree

## Deep Learning

## Project: Build a Traffic Sign Recognition Classifier

In this notebook, a template is provided for you to implement your functionality in stages, which is required to successfully complete this project. If additional code is required that cannot be included in the notebook, be sure that the Python code is successfully imported and included in your submission if necessary. 

> **Note**: Once you have completed all of the code implementations, you need to finalize your work by exporting the iPython Notebook as an HTML document. Before exporting the notebook to html, all of the code cells need to have been run so that reviewers can see the final implementation and output. You can then export the notebook by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission. 

In addition to implementing code, there is a writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) that can be used to guide the writing process. Completing the code template and writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/481/view) for this project.

The [rubric](https://review.udacity.com/#!/rubrics/481/view) contains "Stand Out Suggestions" for enhancing the project beyond the minimum requirements. The stand out suggestions are optional. If you decide to pursue the "stand out suggestions", you can include the code in this Ipython notebook and also discuss the results in the writeup file.


>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

---
## Step 0: Load The Data

In [174]:
%%capture

# Imports
import pickle
import os
import math
import urllib.request
from zipfile import ZipFile
from io import BytesIO

import numpy as np
import pandas as pd

import skimage as ski
import skimage.color
import skimage.transform

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import tensorflow as tf

import sklearn as skl
import sklearn.utils

# Visualizations will be shown in the notebook.
%matplotlib inline

In [175]:
def region_of_interest(img, original_size, original_roi):
    return [
        original_roi[0] / original_size[0] * img.shape[0],
        original_roi[1] / original_size[1] * img.shape[1],
        original_roi[2] / original_size[0] * img.shape[0],
        original_roi[3] / original_size[1] * img.shape[1],
    ]

def pad(img, shape):
    shape = np.max([np.shape(img), shape], axis=0)
    out = np.full(shape, np.nanmean(img))
    out[tuple(slice(0, d) for d in np.shape(img))] = img
    return out

def load_from_pickle(file):
    data = pickle.load(file)
    X, y = data['features'], data['labels']

    assert(len(X) == len(y))

    if 'coords' in data:
        roi = np.zeros_like(data['coords'], dtype=np.float)
        for i in range(len(roi)):
            roi[i] = region_of_interest(data['features'][i], data['sizes'][i], data['coords'][i])
    else:
        roi = []

    return X, y, roi

In [176]:
# Load pickled data

DATA_DIR = 'data'
ARCHIVE_URL = 'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip'

# Repository location of the saved training and testing data
training_file = DATA_DIR + '/train.p'
validation_file = DATA_DIR + '/valid.p'
testing_file = DATA_DIR + '/test.p'

if any(not os.path.exists(file) for file in (training_file, validation_file, testing_file)):
    os.makedirs(DATA_DIR)

    url = urllib.request.urlopen(ARCHIVE_URL)

    with ZipFile(BytesIO(url.read())) as my_zip_file:
        my_zip_file.extractall(DATA_DIR)

with open(training_file, mode='rb') as f:
    X_train, y_train, roi_train = load_from_pickle(f)
with open(validation_file, mode='rb') as f:
    X_valid, y_valid, roi_valid = load_from_pickle(f)
with open(testing_file, mode='rb') as f:
    X_test, y_test, roi_test = load_from_pickle(f)

print("Data successfully loaded.")

Data successfully loaded.


---

## Step 1: Dataset Summary & Exploration

The pickled data is a dictionary with 4 key/value pairs:

- `'features'` is a 4D array containing raw pixel data of the traffic sign images, (num examples, width, height, channels).
- `'labels'` is a 1D array containing the label/class id of the traffic sign. The file `signnames.csv` contains id -> name mappings for each id.
- `'sizes'` is a list containing tuples, (width, height) representing the original width and height the image.
- `'coords'` is a list containing tuples, (x1, y1, x2, y2) representing coordinates of a bounding box around the sign in the image. **THESE COORDINATES ASSUME THE ORIGINAL IMAGE. THE PICKLED DATA CONTAINS RESIZED VERSIONS (32 by 32) OF THESE IMAGES**

Complete the basic data summary below. Use python, numpy and/or pandas methods to calculate the data summary rather than hard coding the results. For example, the [pandas shape method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shape.html) might be useful for calculating some of the summary results. 

### Summary and visualization of the dataset

In [ ]:
def summarize_dataset(train=([],[],[]), valid=([],[],[]), test=([],[],[])):
    (X_train, y_train, roi_train) = train
    (X_valid, y_valid, roi_valid) = valid
    (X_test, y_test, roi_test) = test

    # Assess unique classes and distribution of the training dataset using NumPy
    classes, classes_count = np.unique(y_test, return_counts=True)

    # Number of training examples
    n_train = len(y_train)
    # Number of validation examples
    n_validation = len(y_valid)
    # Number of testing examples.
    n_test = len(y_test)
    # What's the shape of an traffic sign image?
    image_shape = X_test.shape[1:]
    # How many unique classes/labels there are in the dataset.
    n_classes = len(classes)

    # Print basic summary
    print("Number of training examples =", n_train)
    print("Number of validation examples =", n_train)
    print("Number of testing examples =", n_test)
    print("Image data shape =", image_shape)
    print("Number of classes =", n_classes)

    # Plot a histogram, showing the distribution of samples across classes using Pandas
    SIGNNAMES_FILE = 'signnames.csv'
    classes_df = pd.read_csv(SIGNNAMES_FILE, index_col=0)
    classes_df['Training'] = pd.Series(y_train).value_counts().sort_index()
    classes_df['Validation'] = pd.Series(y_valid).value_counts().sort_index()
    classes_df['Testing']  = pd.Series(y_test) .value_counts().sort_index()
    classes_df.plot(kind='bar', stacked=True, figsize=(15,5))


    # create figure (fig), and array of axes (ax)
    ncols = 5
    nrows = (n_classes + ncols - 1) // ncols
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15,15), squeeze=True)

    # plot simple raster image on each sub-plot
    for i, axi in enumerate(ax.flat):
        # i runs from 0 to (nrows*ncols-1)
        if i >= n_classes:
            fig.delaxes(axi)
            continue

        cls_id = classes_df.index[i]
        cls_title = classes_df.iloc[:,0][i]
        cls_count = classes_df.iloc[:,3][i]

        indices = np.where(y_test==cls_id)[0]
        sel_index = np.random.choice(indices)
        img = X_test[sel_index]
        axi.imshow(img)
        axi.set_title("{:02d}: {:20.20s} [{}]".format(cls_id, cls_title, cls_count))

        if len(roi_test) > 0:
            roi = roi_test[sel_index]
            axi.add_patch(Rectangle((roi[0],roi[1]),roi[2]-roi[0],roi[3]-roi[1],linewidth=1,edgecolor='r',facecolor='none'))

    # # one can access the axes by ax[row_id][col_id]
    # # do additional plotting on ax[row_id][col_id] of your choice
    # ax[0][2].plot(xs, 3*ys, color='red', linewidth=3)
    # ax[4][3].plot(ys**2, xs, color='green', linewidth=3)

    plt.tight_layout(pad=1.08)
    plt.show()

    return classes


classes = summarize_dataset(
    (X_train, y_train, roi_train),
    (X_valid, y_valid, roi_valid),
    (X_test, y_test, roi_test)
)
n_classes = len(classes)

Number of training examples = 34799
Number of validation examples = 34799
Number of testing examples = 12630
Image data shape = (32, 32, 3)
Number of classes = 43


----

## Step 2: Design and Test a Model Architecture

Design and implement a deep learning model that learns to recognize traffic signs. Train and test your model on the [German Traffic Sign Dataset](http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset).

The LeNet-5 implementation shown in the [classroom](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) at the end of the CNN lesson is a solid starting point. You'll have to change the number of classes and possibly the preprocessing, but aside from that it's plug and play! 

With the LeNet-5 solution from the lecture, you should expect a validation set accuracy of about 0.89. To meet specifications, the validation set accuracy will need to be at least 0.93. It is possible to get an even higher accuracy, but 0.93 is the minimum for a successful project submission. 

There are various aspects to consider when thinking about this problem:

- Neural network architecture (is the network over or underfitting?)
- Play around preprocessing techniques (normalization, rgb to grayscale, etc)
- Number of examples per label (some have more than others).
- Generate fake data.

Here is an example of a [published baseline model on this problem](http://yann.lecun.com/exdb/publis/pdf/sermanet-ijcnn-11.pdf). It's not required to be familiar with the approach used in the paper but, it's good practice to try to read papers like these.

### Pre-process the Data Set (normalization, grayscale, etc.)

Minimally, the image data should be normalized so that the data has mean zero and equal variance. For image data, `(pixel - 128)/ 128` is a quick way to approximately normalize the data and can be used in this project. 

Other pre-processing steps are optional. You can try different techniques to see if it improves performance. 

Use the code cell (or multiple code cells, if necessary) to implement the first step of your project.

In [ ]:
### Preprocess the data here. It is required to normalize the data. Other preprocessing steps could include 
### converting to grayscale, etc.
### Feel free to use as many code cells as needed.

def preprocess_images(dataset, dataset_roi=None, size=(32,32)):
    # Convert to grayscale
    if dataset.shape[-1] == 3:
        dataset = ski.color.rgb2gray(dataset) * 255


    if not dataset_roi is None:
        for i in range(min(len(dataset), len(dataset_roi))):

            # plt.figure()
            # plt.imshow(dataset[i])
            # plt.gca().add_patch(Rectangle((dataset_roi[i][0],dataset_roi[i][1]),dataset_roi[i][2]-dataset_roi[i][0],dataset_roi[i][3]-dataset_roi[i][1],linewidth=1,edgecolor='r',facecolor='none'))
            # print(dataset_roi[i])

            x1 = max(math.floor(dataset_roi[i][0])-1, 0)
            x2 = min(math.ceil(dataset_roi[i][2])+2, size[0])
            y1 = max(math.floor(dataset_roi[i][1]-1), 0)
            y2 = min(math.ceil(dataset_roi[i][3])+2, size[1])

            cropped = dataset[i,y1:y2,x1:x2]

            # plt.figure()
            # plt.imshow(cropped)

            padded = pad(cropped, (max(cropped.shape),max(cropped.shape)))

            # print(x1,y1,x2,y2)
            # plt.figure()
            # plt.imshow(padded)

            resized = ski.transform.resize(padded, (size[0],size[1]), anti_aliasing=True)
            dataset[i] = resized

            # plt.figure()
            # plt.imshow(resized)


    # Reshape for tensor input
    dataset = dataset.reshape((-1, 32, 32, 1))

    # Normalize image data
    dataset = (dataset - 128) / 128

    return dataset


def color_depth(dataset):
    return dataset.shape[-1] if (not dataset is None and dataset.ndim == 4) else 0


# Preprocess images
X_train = preprocess_images(X_train, roi_train)
X_valid = preprocess_images(X_valid, roi_valid)
X_test = preprocess_images(X_test, roi_test)

# Determine input depth
input_depth = color_depth(X_train)

# TODO: Augment the data


### Model Architecture

In [ ]:
### Define your architecture here.
### Feel free to use as many code cells as needed.

import tensorflow as tf
from tensorflow.contrib.layers import flatten

def conv2d(x, W, b, strides=1, padding='VALID'):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding=padding)
    return tf.nn.bias_add(x, b)


def pool2d(x, k=2):
    return tf.nn.max_pool(
        x,
        ksize=[1, k, k, 1],
        strides=[1, k, k, 1],
        padding='VALID')


def fulcon(x, W, b):
    return tf.add(tf.matmul(x, W), b)


# Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
mu = 0
sigma = 0.1

weights = {
    'w1': tf.Variable(tf.truncated_normal([5, 5, input_depth, 6], mu, sigma)),
    'w2': tf.Variable(tf.truncated_normal([5, 5, 6, 16], mu, sigma)),
    'w3': tf.Variable(tf.truncated_normal([400, 120], mu, sigma)),
    'w4': tf.Variable(tf.truncated_normal([120, 84], mu, sigma)),
    'w5': tf.Variable(tf.truncated_normal([84, n_classes], mu, sigma))
}

biases = {
    'b1': tf.Variable(tf.zeros(6)),
    'b2': tf.Variable(tf.zeros(16)),
    'b3': tf.Variable(tf.zeros(120)),
    'b4': tf.Variable(tf.zeros(84)),
    'b5': tf.Variable(tf.zeros(n_classes))
}

keep_prob = tf.placeholder_with_default(0.7, ())

def LeNet(x):
    # Layer 1: Convolutional. Input = 32x32xinput_depth. Output = 28x28x6.
    conv1 = conv2d(x, weights['w1'], biases['b1'])
    # Activation.
    conv1 = tf.nn.tanh(conv1)
    # Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = pool2d(conv1, 2)

    # Layer 2: Convolutional. Output = 10x10x16.
    conv2 = conv2d(conv1, weights['w2'], biases['b2'])
    # Activation.
    conv2 = tf.nn.tanh(conv2)
    # Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = pool2d(conv2, 2)

    # Flatten. Input = 5x5x16. Output = 400.
    flat = tf.contrib.layers.flatten(conv2)
    flat = tf.nn.dropout(flat, rate=1.0-keep_prob)

    # Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1 = fulcon(flat, weights['w3'], biases['b3'])
    # Activation.
    fc1 = tf.nn.tanh(fc1)
    # Dropout.
    fc1 = tf.nn.dropout(fc1, rate=1.0-keep_prob)

    # Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2 = fulcon(fc1, weights['w4'], biases['b4'])
    # Activation.
    fc2 = tf.nn.tanh(fc2)
    # Dropout.
    fc2 = tf.nn.dropout(fc2, rate=1.0-keep_prob)

    # Layer 5: Fully Connected. Input = 84. Output = n_classes.
    logits = fulcon(fc2, weights['w5'], biases['b5'])

    return logits



### Train, Validate and Test the Model

A validation set can be used to assess how well the model is performing. A low accuracy on the training and validation
sets imply underfitting. A high accuracy on the training set but low accuracy on the validation set implies overfitting.

In [ ]:
### Train your model here.
### Calculate and report the accuracy on the training and validation set.
### Once a final model architecture is selected, 
### the accuracy on the test set should be calculated and reported as well.
### Feel free to use as many code cells as needed.


x = tf.placeholder(tf.float32, (None, 32, 32, input_depth))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, n_classes)

BETA = 0.01
RATE = 0.002
EPOCHS = 25
BATCH_SIZE = 256

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
regularizer = (tf.nn.l2_loss(weights['w5']))
loss_operation = tf.reduce_mean(loss_operation + BETA * regularizer)
optimizer = tf.train.AdamOptimizer(learning_rate = RATE)
training_operation = optimizer.minimize(loss_operation)


In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.0})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples


In [ ]:
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)

    epochs = []
    taccs = []
    vaccs = []

    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = skl.utils.shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})

        training_accuracy = evaluate(X_train, y_train)
        validation_accuracy = evaluate(X_valid, y_valid)
        print("EPOCH {} ...".format(i+1))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print()

        epochs.append(i+1)
        taccs.append(training_accuracy)
        vaccs.append(validation_accuracy)

    saver.save(sess, './lenet')
    print("Model saved")


In [ ]:
if epochs:
    fig = plt.figure()
    plt.title('Model Accuracy')
    plt.plot(epochs, taccs, '.-b', label='Training')
    plt.plot(epochs, vaccs, '.-r', label='Validation')
    plt.legend(loc='lower right')
    fig.gca().xaxis.get_major_locator().set_params(integer=True)
    fig.gca().set_xticks(np.arange(0, EPOCHS+1, EPOCHS//10))
    plt.grid()
    plt.xlabel('Epoch')


with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    test_accuracy = evaluate(X_test, y_test)
    print("Test Accuracy = {:.3f}".format(test_accuracy))


---

## Step 3: Test a Model on New Images

To give yourself more insight into how your model is working, download at least five pictures of German traffic signs from the web and use your model to predict the traffic sign type.

You may find `signnames.csv` useful as it contains mappings from the class id (integer) to the actual sign name.

### Load and Output the Images

In [ ]:
### Load the images and plot them here.
### Feel free to use as many code cells as needed.

from skimage.io import imread
from skimage.util import img_as_ubyte
from skimage.transform import rescale

ARCHIVE_URL = 'https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip'

# Repository location of the saved training and testing data
new_file = DATA_DIR + '/new.p'

if True or not os.path.exists(new_file):
    os.makedirs(DATA_DIR, exist_ok=True)

    url = urllib.request.urlopen(ARCHIVE_URL)

    new_features = []
    new_labels = []
    new_sizes = []
    new_coords = []

    with ZipFile(BytesIO(url.read())) as zip_file:
        for file_path in zip_file.namelist():
            file_path = os.path.normpath(file_path)
            cls_dir, filename = ([None]*2+file_path.split(os.sep))[-2::]

            if not (cls_dir and cls_dir.isdigit()):
                continue

            cls = int(cls_dir)
            if not cls in classes:
                continue

            if filename.endswith('.csv'):
                csv_df = pd.read_csv(zip_file.open(file_path), ';')
                for row in csv_df.itertuples():
                    new_sizes.append((row[2], row[3]))
                    new_coords.append((row[4], row[5], row[6], row[7]))

            elif filename.endswith('.ppm'):
                img = imread(zip_file.open(file_path), as_gray=False)
                img = img_as_ubyte(img)

                # scale = 32.0 / max(img.shape)
                # if scale < 1.0:
                #     img = rescale(img, scale, anti_aliasing=True, preserve_range=True)
                #
                # if img.shape != (32, 32):
                #     img = pad(img, (32,32))
                #
                # img = img.astype(np.uint8)

                new_features.append(img)
                new_labels.append(cls)

                # plt.figure()
                # plt.imshow(img)
                # print(cls, img.shape)

    if new_features:
        new_data = {
            'features': np.array(new_features),
            'labels': np.array(new_labels)
        }

        if len(new_coords) and len(new_sizes):
            new_data['coords'] = np.array(new_coords)
            new_data['sizes'] = np.array(new_sizes)

        with open(new_file, mode='wb') as f:
            pickle.dump(new_data, f)

In [ ]:
with open(new_file, mode='rb') as f:
    X_new, y_new, roi_new = load_from_pickle(f)

summarize_dataset(test=(X_new, y_new, roi_new))


### Predict the Sign Type for Each Image

In [ ]:
### Run the predictions here and use the model to output the prediction for each image.
### Make sure to pre-process the images with the same pre-processing pipeline used earlier.
### Feel free to use as many code cells as needed.

# Preprocess images
X_new = preprocess_images(X_new, roi_new)

# Determine input depth
input_depth = color_depth(X_new)


### Analyze Performance

In [ ]:
### Calculate the accuracy for these 5 new images. 
### For example, if the model predicted 1 out of 5 signs correctly, it's 20% accurate on these new images.

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    new_accuracy = evaluate(X_new, y_new)
    print("New Data Accuracy = {:.3f}".format(new_accuracy))


selection = np.random.choice(len(X_new), 5, replace=False)
X_new = X_new[selection]
y_new = y_new[selection]

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    new_accuracy = evaluate(X_new, y_new)
    print("Random 5 Accuracy = {:.3f}".format(new_accuracy))


### Output Top 5 Softmax Probabilities For Each Image Found on the Web

For each of the new images, print out the model's softmax probabilities to show the **certainty** of the model's predictions (limit the output to the top 5 probabilities for each image). [`tf.nn.top_k`](https://www.tensorflow.org/versions/r0.12/api_docs/python/nn.html#top_k) could prove helpful here. 


In [ ]:
### Print out the top five softmax probabilities for the predictions on the German traffic sign images found on the web. 
### Feel free to use as many code cells as needed.

topk_operation = tf.nn.top_k(tf.nn.softmax(logits), k=5)

with tf.Session() as sess:
    saver.restore(sess, tf.train.latest_checkpoint('.'))

    top5_softmax = sess.run(topk_operation, feed_dict={x: X_new, y: y_new, keep_prob: 1.0})


for i in range(len(X_new)):
    plt.figure()
    plt.imshow(X_new[i], cmap="gray")

    top5 = dict(zip(top5_softmax.indices[i], top5_softmax.values[i]))
    print(top5)



### Project Writeup

Once you have completed the code implementation, document your results in a project writeup using this [template](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/writeup_template.md) as a guide. The writeup can be in a markdown or pdf file. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  \n",
    "**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.

---

## Step 4 (Optional): Visualize the Neural Network's State with Test Images

 This Section is not required to complete but acts as an additional excersise for understaning the output of a neural network's weights. While neural networks can be a great learning device they are often referred to as a black box. We can understand what the weights of a neural network look like better by plotting their feature maps. After successfully training your neural network you can see what it's feature maps look like by plotting the output of the network's weight layers in response to a test stimuli image. From these plotted feature maps, it's possible to see what characteristics of an image the network finds interesting. For a sign, maybe the inner network feature maps react with high activation to the sign's boundary outline or to the contrast in the sign's painted symbol.

 Provided for you below is the function code that allows you to get the visualization output of any tensorflow weight layer you want. The inputs to the function should be a stimuli image, one used during training or a new one you provided, and then the tensorflow variable name that represents the layer's state during the training process, for instance if you wanted to see what the [LeNet lab's](https://classroom.udacity.com/nanodegrees/nd013/parts/fbf77062-5703-404e-b60c-95b78b2f3f9e/modules/6df7ae49-c61c-4bb2-a23e-6527e69209ec/lessons/601ae704-1035-4287-8b11-e2c2716217ad/concepts/d4aca031-508f-4e0b-b493-e7b706120f81) feature maps looked like for it's second convolutional layer you could enter conv2 as the tf_activation variable.

For an example of what feature map outputs look like, check out NVIDIA's results in their paper [End-to-End Deep Learning for Self-Driving Cars](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/) in the section Visualization of internal CNN State. NVIDIA was able to show that their network's inner weights had high activations to road boundary lines by comparing feature maps from an image with a clear path to one without. Try experimenting with a similar test to show that your trained network's weights are looking for interesting features, whether it's looking at differences in feature maps from images with or without a sign, or even what feature maps look like in a trained network vs a completely untrained one on the same sign image.

<figure>
 <img src="visualize_cnn.png" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above)</p> 
 </figcaption>
</figure>
 <p></p> 


In [ ]:
### Visualize your network's feature maps here.
### Feel free to use as many code cells as needed.

# image_input: the test image being fed into the network to produce the feature maps
# tf_activation: should be a tf variable name used during your training procedure that represents the calculated state of a specific weight layer
# activation_min/max: can be used to view the activation contrast in more detail, by default matplot sets min and max to the actual min and max values of the output
# plt_num: used to plot out multiple different weight feature map sets on the same block, just extend the plt number for each new feature map entry

def outputFeatureMap(image_input, tf_activation, activation_min=-1, activation_max=-1 ,plt_num=1):
    # Here make sure to preprocess your image_input in a way your network expects
    # with size, normalization, ect if needed
    # image_input =
    # Note: x should be the same name as your network's tensorflow data placeholder variable
    # If you get an error tf_activation is not defined it may be having trouble accessing the variable from inside a function
    activation = tf_activation.eval(session=sess,feed_dict={x : image_input})
    featuremaps = activation.shape[3]
    plt.figure(plt_num, figsize=(15,15))
    for featuremap in range(featuremaps):
        plt.subplot(6,8, featuremap+1) # sets the number of feature maps to show on each row and column
        plt.title('FeatureMap ' + str(featuremap)) # displays the feature map number
        if activation_min != -1 & activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, vmax=activation_max, cmap="gray")
        elif activation_max != -1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmax=activation_max, cmap="gray")
        elif activation_min !=-1:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", vmin=activation_min, cmap="gray")
        else:
            plt.imshow(activation[0,:,:, featuremap], interpolation="nearest", cmap="gray")